In [7]:
import pandas as pd
import numpy as np
import csv
import json

### Postcode

In [8]:
# we extract postcode in Victoria only
tmp_lst = []
with open('../data/raw/postcode.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst[1:], columns=tmp_lst[0])

df = df[df['state']=='VIC']

# we select some important columns for this dataset, such as SA2 code, SA2 name, latitude, longitude and so on
data = df[['id','postcode','locality','state','long','lat','SA2_MAINCODE_2016','SA2_NAME_2016']] 

# remove duplicates based on SA2 code
data = data.drop_duplicates(['SA2_MAINCODE_2016'])

data.to_csv('../data/curated/postcode_vic.csv')

### Property

In [9]:
# for property json file, we want to convert it into data frame and save as csv
f = open('../data/raw/example.json')
data = json.load(f)
df = pd.DataFrame.from_dict(data, orient='index')
print(len(df))
df.to_csv('../data/curated/property.csv')

15226


### Unemployment

In [10]:
# we want information in Victoria only and only analyse unemployment rate
tmp_lst = []
with open('../data/raw/unemployment.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst[2:], columns=tmp_lst[1])

df = df[df['Data Item'] == 'Smoothed unemployment rate (%)']

vic_code = []
with open('../data/curated/postcode_vic.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        vic_code.append(row[7])
vic_code = vic_code[1:]

new = df['SA2 Code (2016 ASGS)'].isin(vic_code)
data = df[new]

data.to_csv('../data/curated/unemployment.csv')


### School

In [11]:
tmp_lst = []
with open('../data/raw/school.csv', mode='r', encoding = "ISO-8859-1") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst, columns=header)

df = df[['Education_Sector','School_Name','School_Type','Address_Town','Address_State','Address_Postcode','X','Y']]

df.to_csv('../data/curated/school.csv')

### Boundary

In [12]:
import geopandas as gpd

In [13]:
sf = gpd.read_file("../data/curated/sa2/SA2_2021_AUST_GDA94.shp")
gdf = gpd.GeoDataFrame(sf)
gdf = gdf[gdf['STE_NAME21']=='Victoria']

gdf = gdf[['SA2_NAME21','SA2_CODE21', 'geometry']]
gdf.to_file('../data/curated/boundary.shp')